In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.examples.tutorials.mnist import input_data


In [2]:
# 加载了数据
mnist=input_data.read_data_sets('./',one_hot=True)

Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [3]:
# 创建变量和占位符
X=tf.compat.v1.placeholder(dtype=tf.float64,shape=[None,784])
y=tf.compat.v1.placeholder(dtype=tf.float64,shape=[None,10])


In [4]:
# 卷积核在神经网络中是变量
# 变量生成方法
def gen_v(shape):
        return tf.Variable(
            initial_value=tf.random.normal(dtype=tf.float64,shape=shape,stddev=0.1),
            dtype=tf.float64)

In [5]:
# 定义方法，完成卷积操作

def conv(input_data,filters_):
    return tf.nn.conv2d(input=input_data,filters=filters_,strides=[1,1,1,1],padding="SAME")

In [6]:
# 定义池化操作
def pool(input_data):
    return tf.nn.max_pool(input=input_data,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [8]:
X

<tf.Tensor 'Placeholder:0' shape=(None, 784) dtype=float64>

In [9]:
# 第一层卷积
input_data1=tf.reshape(X,shape=[-1,28,28,1])
# 卷积核
filter1=gen_v(shape=[3,3,1,64])
# 偏差 bias
b1=gen_v(shape=[64])
conv1=conv(input_data1,filter1)+b1
# 池化
pool1=pool(conv1)
# 激活 函数
active1=tf.nn.relu(pool1)
active1

<tf.Tensor 'Relu:0' shape=(None, 14, 14, 64) dtype=float64>

In [10]:
# 第二层卷积

# 使用的是第一层卷积的数据

filter2=gen_v(shape=[3,3,64,64])

b2=gen_v(shape=[64])

conv2=conv(active1,filter2)+b2

# 池化

pool2=pool(conv2)

active2=tf.nn.sigmoid(pool2)
active2.shape

TensorShape([None, 7, 7, 64])

In [11]:
# 全连接层
# 1024个连接 1024个方程 1024个神经元

fc_w=gen_v(shape=[7*7*64,1024])
fc_b=gen_v(shape=[1024])

conn=tf.matmul(tf.reshape(active2,shape=[-1,7*7*64]),fc_w)+fc_b

In [12]:
# dropout 仿止过拟合
kp=tf.compat.v1.placeholder(dtype=tf.float64,shape=None)

dropout_=tf.nn.dropout(conn,rate=kp)
dropout_.shape

TensorShape([None, 1024])

In [13]:
# 输出层
# 10个类别 0 ~ 9
out_w=gen_v(shape=[1024,10])

out_b=gen_v(shape=[10])

out=tf.matmul(dropout_,out_w)+out_b
out



<tf.Tensor 'add_3:0' shape=(None, 10) dtype=float64>

In [14]:
prob=tf.nn.softmax(out)
# 预测的概率 prob_ 非真实分布
prob

<tf.Tensor 'Softmax:0' shape=(None, 10) dtype=float64>

In [15]:
# 正实的概率 
y

<tf.Tensor 'Placeholder_1:0' shape=(None, 10) dtype=float64>

In [16]:
# 交叉熵
cost=tf.reduce_mean(tf.reduce_sum(y*tf.math.log(1/prob),axis=-1))
cost

<tf.Tensor 'Mean:0' shape=() dtype=float64>

In [17]:
cost2=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prob))
cost2

<tf.Tensor 'Mean_1:0' shape=() dtype=float64>

In [18]:
adam=tf.compat.v1.train.AdamOptimizer()

In [19]:
optimizer=adam.minimize(cost)
optimizer

<tf.Operation 'Adam' type=NoOp>

In [20]:
saver=tf.compat.v1.train.Saver()

In [20]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)  
config_=tf.compat.v1.ConfigProto(gpu_options=gpu_options,log_device_placement=True,allow_soft_placement=True)

with tf.compat.v1.Session(config=config_) as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    for i in range(10):
        c=0
        for j in range(100):
            x_train,y_train=mnist.train.next_batch(batch_size=550)
            optimizer_,cost_=sess.run(fetches=[optimizer,cost],feed_dict={X:x_train,y:y_train,kp:0.5})
            c+=cost_/100 
            print("里层循环次数：%d ，每次的损失: %0.4f"%(j,cost_))
        print("------执行的次数是: %d 损失函数是: %0.4f------"%(i,c))
        saver.save(sess,save_path="./cnn/model",global_step=i)
        

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5

里层循环次数：0 ，每次的损失: 24.7065
里层循环次数：1 ，每次的损失: 34.8216
里层循环次数：2 ，每次的损失: 46.9154
里层循环次数：3 ，每次的损失: 58.0738
里层循环次数：4 ，每次的损失: 55.3210
里层循环次数：5 ，每次的损失: 57.4655
里层循环次数：6 ，每次的损失: 43.9547
里层循环次数：7 ，每次的损失: 33.6276
里层循环次数：8 ，每次的损失: 32.2136
里层循环次数：9 ，每次的损失: 23.3635
里层循环次数：10 ，每次的损失: 25.5279
里层循环次数：11 ，每次的损失: 19.6013
里层循环次数：12 ，每次的损失: 21.2818
里层循环次数：13 ，每次的损失: 19.1535
里层循环次数：14 ，每次的损失: 14.1598
里层循环次数：15 ，每次的损失: 14.9497
里层循环次数：16 ，每次的损失: 12.6923
里层循环次数：17 ，每次的损失: 15.5546
里层循环次数：18 ，每次的损失: 12.4116
里层循环次数：19 ，每次的损失: 9.2828
里层循环次数：20 ，每次的损失: 12.0515
里层循环次数：21 ，每次的损失: 11.4726
里层循环次数：22 ，每次的损失: 8.7539
里层循环次数：23 ，每次的损失: 9.0838
里层循环次数：24 ，每次的损失: 6.5192
里层循环次数：25 ，每次的损失: 7.4514
里层循环次数：26 ，每次的损失: 4.7523
里层循环次数：27 ，每次的损失: 6.7221
里层循环次数：28 ，每次的损失: 6.4591
里层循环次数：29 ，每次的损失: 6.1634
里层循环次数：30 ，每次的损失: 5.1489
里层循环次数：31 ，每次的损失: 4.0542
里层循环次数：32 ，每次的损失: 5.5147
里层循环

里层循环次数：18 ，每次的损失: 0.2330
里层循环次数：19 ，每次的损失: 0.3070
里层循环次数：20 ，每次的损失: 0.2560
里层循环次数：21 ，每次的损失: 0.2193
里层循环次数：22 ，每次的损失: 0.2989
里层循环次数：23 ，每次的损失: 0.2792
里层循环次数：24 ，每次的损失: 0.2856
里层循环次数：25 ，每次的损失: 0.2286
里层循环次数：26 ，每次的损失: 0.2787
里层循环次数：27 ，每次的损失: 0.2054
里层循环次数：28 ，每次的损失: 0.3716
里层循环次数：29 ，每次的损失: 0.2432
里层循环次数：30 ，每次的损失: 0.1972
里层循环次数：31 ，每次的损失: 0.3094
里层循环次数：32 ，每次的损失: 0.2409
里层循环次数：33 ，每次的损失: 0.2602
里层循环次数：34 ，每次的损失: 0.3100
里层循环次数：35 ，每次的损失: 0.2419
里层循环次数：36 ，每次的损失: 0.2262
里层循环次数：37 ，每次的损失: 0.2875
里层循环次数：38 ，每次的损失: 0.2214
里层循环次数：39 ，每次的损失: 0.2267
里层循环次数：40 ，每次的损失: 0.2287
里层循环次数：41 ，每次的损失: 0.2418
里层循环次数：42 ，每次的损失: 0.2920
里层循环次数：43 ，每次的损失: 0.2339
里层循环次数：44 ，每次的损失: 0.3013
里层循环次数：45 ，每次的损失: 0.2121
里层循环次数：46 ，每次的损失: 0.2578
里层循环次数：47 ，每次的损失: 0.2258
里层循环次数：48 ，每次的损失: 0.3026
里层循环次数：49 ，每次的损失: 0.2638
里层循环次数：50 ，每次的损失: 0.3152
里层循环次数：51 ，每次的损失: 0.2758
里层循环次数：52 ，每次的损失: 0.2027
里层循环次数：53 ，每次的损失: 0.3098
里层循环次数：54 ，每次的损失: 0.2644
里层循环次数：55 ，每次的损失: 0.2167
里层循环次数：56 ，每次的损失: 0.2804
里层循环次数：57 ，每次的损失: 0.2955


里层循环次数：30 ，每次的损失: 0.1823
里层循环次数：31 ，每次的损失: 0.1691
里层循环次数：32 ，每次的损失: 0.0929
里层循环次数：33 ，每次的损失: 0.1720
里层循环次数：34 ，每次的损失: 0.1518
里层循环次数：35 ，每次的损失: 0.1462
里层循环次数：36 ，每次的损失: 0.2180
里层循环次数：37 ，每次的损失: 0.1891
里层循环次数：38 ，每次的损失: 0.2152
里层循环次数：39 ，每次的损失: 0.1856
里层循环次数：40 ，每次的损失: 0.1752
里层循环次数：41 ，每次的损失: 0.1725
里层循环次数：42 ，每次的损失: 0.1908
里层循环次数：43 ，每次的损失: 0.1252
里层循环次数：44 ，每次的损失: 0.1424
里层循环次数：45 ，每次的损失: 0.2337
里层循环次数：46 ，每次的损失: 0.1579
里层循环次数：47 ，每次的损失: 0.1473
里层循环次数：48 ，每次的损失: 0.1191
里层循环次数：49 ，每次的损失: 0.1552
里层循环次数：50 ，每次的损失: 0.1684
里层循环次数：51 ，每次的损失: 0.1427
里层循环次数：52 ，每次的损失: 0.1519
里层循环次数：53 ，每次的损失: 0.1495
里层循环次数：54 ，每次的损失: 0.1637
里层循环次数：55 ，每次的损失: 0.1482
里层循环次数：56 ，每次的损失: 0.2255
里层循环次数：57 ，每次的损失: 0.1315
里层循环次数：58 ，每次的损失: 0.1192
里层循环次数：59 ，每次的损失: 0.1520
里层循环次数：60 ，每次的损失: 0.1972
里层循环次数：61 ，每次的损失: 0.1771
里层循环次数：62 ，每次的损失: 0.1844
里层循环次数：63 ，每次的损失: 0.1518
里层循环次数：64 ，每次的损失: 0.1657
里层循环次数：65 ，每次的损失: 0.1562
里层循环次数：66 ，每次的损失: 0.1938
里层循环次数：67 ，每次的损失: 0.1867
里层循环次数：68 ，每次的损失: 0.1391
里层循环次数：69 ，每次的损失: 0.1574


里层循环次数：55 ，每次的损失: 0.0986
里层循环次数：56 ，每次的损失: 0.1429
里层循环次数：57 ，每次的损失: 0.1447
里层循环次数：58 ，每次的损失: 0.0666
里层循环次数：59 ，每次的损失: 0.0735
里层循环次数：60 ，每次的损失: 0.1017
里层循环次数：61 ，每次的损失: 0.0791
里层循环次数：62 ，每次的损失: 0.0901
里层循环次数：63 ，每次的损失: 0.1196
里层循环次数：64 ，每次的损失: 0.0953
里层循环次数：65 ，每次的损失: 0.1684
里层循环次数：66 ，每次的损失: 0.1231
里层循环次数：67 ，每次的损失: 0.1409
里层循环次数：68 ，每次的损失: 0.1201
里层循环次数：69 ，每次的损失: 0.0630
里层循环次数：70 ，每次的损失: 0.1677
里层循环次数：71 ，每次的损失: 0.1274
里层循环次数：72 ，每次的损失: 0.1158
里层循环次数：73 ，每次的损失: 0.0673
里层循环次数：74 ，每次的损失: 0.1121
里层循环次数：75 ，每次的损失: 0.1683
里层循环次数：76 ，每次的损失: 0.1213
里层循环次数：77 ，每次的损失: 0.1502
里层循环次数：78 ，每次的损失: 0.1730
里层循环次数：79 ，每次的损失: 0.1139
里层循环次数：80 ，每次的损失: 0.1046
里层循环次数：81 ，每次的损失: 0.0716
里层循环次数：82 ，每次的损失: 0.1175
里层循环次数：83 ，每次的损失: 0.1497
里层循环次数：84 ，每次的损失: 0.1536
里层循环次数：85 ，每次的损失: 0.0788
里层循环次数：86 ，每次的损失: 0.1290
里层循环次数：87 ，每次的损失: 0.1831
里层循环次数：88 ，每次的损失: 0.1139
里层循环次数：89 ，每次的损失: 0.1296
里层循环次数：90 ，每次的损失: 0.0727
里层循环次数：91 ，每次的损失: 0.1254
里层循环次数：92 ，每次的损失: 0.1028
里层循环次数：93 ，每次的损失: 0.1304
里层循环次数：94 ，每次的损失: 0.0716


In [21]:
# from numba import cuda

# cuda.select_device(0) # 选择GPU设备
# cuda.close(

In [22]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess,"./cnn/model-8")
    x_test,y_test=mnist.test.next_batch(batch_size=2000)
    res=sess.run(prob,feed_dict={X:x_test,kp:0.999})
    y_test=y_test.argmax(axis=-1)
    res=res.argmax(axis=-1)
    print((y_test==res).mean())
    print(y_test.shape)
    print(res.shape)
    
    
    

INFO:tensorflow:Restoring parameters from ./cnn/model-8
0.114
(2000,)
(2000,)
